In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os
import matplotlib.pyplot as plt
from pathlib import Path

# ================= CONFIGURATION =================
base_path = r"D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2"
raw_data_dir = os.path.join(base_path, "raw_data")
output_dir = os.path.join(base_path, "raw_labelled_results")

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# ================= CHANNEL INFORMATION FROM PAPER =================
# From paper Table 2 (0-based indexing for Python)
paper_channels = {
    0: 'Fp1', 1: 'Fp2', 2: 'Pz', 3: 'P3', 4: 'P4', 5: 'F7', 6: 'F8', 7: 'FC2', 
    8: 'FC3', 9: 'FC4', 10: 'FT7', 11: 'FT8', 12: 'Cz', 13: 'C3', 14: 'C4',
    15: 'T3', 16: 'T4', 17: 'CPz', 18: 'CP3', 19: 'CP4', 20: 'TP7', 21: 'TP8',
    22: 'Pz', 23: 'P3', 24: 'P4', 25: 'T5', 26: 'T6', 27: 'Oz', 28: 'O1',
    29: 'O2', 30: 'HEOL', 31: 'VEOR', 32: 'Marker'
}

# Classify channels
channels_all = [paper_channels[i] for i in range(33)]
channels_eeg = [paper_channels[i] for i in range(30)]  # First 30 are EEG
channels_eog = [paper_channels[30], paper_channels[31]]  # HEOL, VEOR
channels_marker = [paper_channels[32]]  # Marker channel

print(f"Channels All: {len(channels_all)}")
print(f"Channels EEG: {len(channels_eeg)}")
print(f"Channels EOG: {len(channels_eog)}")
print(f"Channels Marker: {len(channels_marker)}")

# ================= FUNCTION TO PROCESS ONE SUBJECT =================
def process_raw_mat_file(subject_id, raw_data_dir, output_dir, sampling_rate=500):
    """
    Process raw .mat file for one subject and save in the specified format
    
    Parameters:
    -----------
    subject_id : str
        Subject ID (e.g., '01', '02')
    raw_data_dir : str
        Directory containing raw .mat files
    output_dir : str
        Output directory for saved results
    sampling_rate : int
        Sampling rate in Hz (500 from paper)
    
    Returns:
    --------
    result_dict : dict
        Dictionary with all required keys
    """
    
    print(f"\nProcessing subject: {subject_id}")
    
    # Construct file path
    mat_filename = f"sub-{subject_id}_task-motor-imagery_eeg.mat"
    mat_path = os.path.join(raw_data_dir, mat_filename)
    
    if not os.path.exists(mat_path):
        print(f"  ❌ File not found: {mat_path}")
        return None
    
    try:
        # Load .mat file
        data = loadmat(mat_path)
        eeg_struct = data['eeg'][0, 0]
        
        # Extract data and labels
        rawdata = eeg_struct['rawdata']  # Shape: (40, 33, 4000)
        labels = eeg_struct['label'].flatten()  # Shape: (40,)
        
        print(f"  ✓ Loaded data: {rawdata.shape}")
        print(f"  ✓ Labels: {labels.shape}")
        
        # Verify dimensions
        if rawdata.shape != (40, 33, 4000):
            print(f"  ⚠️ Unexpected shape: {rawdata.shape}, expected (40, 33, 4000)")
        
        # ================= EXTRACT MOTOR IMAGERY DATA ONLY =================
        # According to the paper: 2s instruction + 4s MI + 2s break = 8s total per trial
        # With 500Hz sampling rate: 4000 samples per trial
        # We want to remove first 2s (instruction) and last 2s (break)
        # Keep only the middle 4s of Motor Imagery
        
        samples_per_second = sampling_rate  # 500 samples per second
        instruction_samples = 2 * samples_per_second  # First 2 seconds (1000 samples)
        mi_samples = 4 * samples_per_second  # Middle 4 seconds (2000 samples)
        break_samples = 2 * samples_per_second  # Last 2 seconds (1000 samples)
        
        # Verify total samples
        total_samples_per_trial = instruction_samples + mi_samples + break_samples
        print(f"  ✓ Samples per trial breakdown:")
        print(f"    - Instruction (first 2s): {instruction_samples} samples")
        print(f"    - Motor Imagery (middle 4s): {mi_samples} samples")
        print(f"    - Break (last 2s): {break_samples} samples")
        print(f"    - Total: {total_samples_per_trial} samples")
        
        # Extract only the Motor Imagery portion (samples 1000:3000 for each trial)
        # This removes the first 2s (instruction) and last 2s (break)
        mi_start_sample = instruction_samples
        mi_end_sample = instruction_samples + mi_samples
        
        # Extract Motor Imagery data for all channels
        segments_mi_all = rawdata[:, :, mi_start_sample:mi_end_sample]
        
        # Extract only EEG channels for Motor Imagery
        segments_mi_eeg = rawdata[:, :30, mi_start_sample:mi_end_sample]
        
        print(f"  ✓ Original data shape: {rawdata.shape}")
        print(f"  ✓ MI data shape (all channels): {segments_mi_all.shape}")
        print(f"  ✓ MI data shape (EEG only): {segments_mi_eeg.shape}")
        
        # Create event_info DataFrame for Motor Imagery only
        event_info = []
        for trial_idx in range(40):
            label = labels[trial_idx]
            hand = "left" if label == 1 else "right"
            
            event_info.append({
                'trial_idx': trial_idx,
                'onset_sec': trial_idx * 8.0 + 2.0,  # Each trial starts 2s later due to instruction removal
                'duration_sec': 4.0,  # MI duration
                'label': label,
                'hand': hand,
                'condition': 'motor_imagery',
                'sample_start': mi_start_sample,
                'sample_end': mi_end_sample
            })
        
        event_info_df = pd.DataFrame(event_info)
        
        # Create result dictionary with MI data only
        result_dict = {
            'segments_mi_all': segments_mi_all.astype(np.float32),  # Motor Imagery only, all channels
            'segments_mi_eeg': segments_mi_eeg.astype(np.float32),  # Motor Imagery only, EEG channels
            'labels': labels.astype(np.int32),
            'channels_all': channels_all,
            'channels_eeg': channels_eeg,
            'channels_eog': channels_eog,
            'channels_marker': channels_marker,
            'sampling_rate': sampling_rate,
            'subject_id': f"sub-{subject_id}",
            'event_info': event_info_df.to_dict('records'),  # Save as list of dicts for JSON compatibility
            'mi_info': {
                'samples_removed_start': instruction_samples,
                'samples_removed_end': break_samples,
                'mi_duration_samples': mi_samples,
                'mi_duration_seconds': 4.0
            }
        }
        
        # Save to .npz file
        output_path = os.path.join(output_dir, f"sub-{subject_id}_motor_imagery.npz")
        np.savez_compressed(output_path, **result_dict)
        
        print(f"  ✅ Saved to: {output_path}")
        
        # Print summary
        left_count = np.sum(labels == 1)
        right_count = np.sum(labels == 2)
        print(f"  📊 Summary: {left_count} left hand, {right_count} right hand trials")
        
        return result_dict
        
    except Exception as e:
        print(f"  ❌ Error processing {subject_id}: {e}")
        return None

# ================= PROCESS ALL SUBJECTS =================
def process_all_subjects(raw_data_dir, output_dir, max_subjects=None):
    """
    Process all subjects in the raw_data directory
    
    Parameters:
    -----------
    raw_data_dir : str
        Directory containing raw .mat files
    output_dir : str
        Output directory for saved results
    max_subjects : int or None
        Maximum number of subjects to process (for testing)
    """
    
    # Get all .mat files
    mat_files = [f for f in os.listdir(raw_data_dir) if f.endswith('.mat')]
    
    # Sort by subject number
    mat_files.sort(key=lambda x: int(x.split('-')[1].split('_')[0]))
    
    if max_subjects:
        mat_files = mat_files[:max_subjects]
    
    print(f"Found {len(mat_files)} raw .mat files")
    print(f"Output directory: {output_dir}")
    
    all_results = {}
    
    for mat_file in mat_files:
        # Extract subject ID
        subject_id = mat_file.split('-')[1].split('_')[0]  # Get '01' from 'sub-01_task-...'
        
        # Process this subject
        result = process_raw_mat_file(subject_id, raw_data_dir, output_dir)
        
        if result:
            all_results[subject_id] = result
    
    return all_results

# ================= VERIFICATION FUNCTION =================
def verify_saved_file(subject_id, output_dir):
    """
    Verify that the saved file contains all required keys
    """
    file_path = os.path.join(output_dir, f"sub-{subject_id}_motor_imagery.npz")
    
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return False
    
    try:
        # Load the file
        data = np.load(file_path, allow_pickle=True)
        
        print(f"\nVerifying {os.path.basename(file_path)}:")
        print("-" * 50)
        
        # Check all required keys
        required_keys = [
            'segments_mi_all', 'segments_mi_eeg', 'labels', 'channels_all',
            'channels_eeg', 'channels_eog', 'channels_marker',
            'sampling_rate', 'subject_id', 'event_info', 'mi_info'
        ]
        
        all_present = True
        for key in required_keys:
            if key in data:
                value = data[key]
                if hasattr(value, 'shape'):
                    print(f"✓ {key}: shape={value.shape}, dtype={value.dtype}")
                elif isinstance(value, dict):
                    print(f"✓ {key}: {value}")
                else:
                    print(f"✓ {key}: {type(value)}")
            else:
                print(f"✗ {key}: MISSING!")
                all_present = False
        
        # Print sample information
        if 'labels' in data:
            labels = data['labels']
            print(f"\nLabel distribution:")
            print(f"  Left hand (1): {np.sum(labels == 1)}")
            print(f"  Right hand (2): {np.sum(labels == 2)}")
        
        if 'segments_mi_eeg' in data:
            segments = data['segments_mi_eeg']
            print(f"\nMotor Imagery Segment information:")
            print(f"  Shape: {segments.shape}")
            print(f"  Trials: {segments.shape[0]}")
            print(f"  Channels: {segments.shape[1]}")
            print(f"  Samples per trial: {segments.shape[2]}")
            print(f"  Duration per trial: {segments.shape[2] / 500:.1f}s")
            print(f"  Memory: {segments.nbytes / 1024 / 1024:.2f} MB")
        
        data.close()
        return all_present
        
    except Exception as e:
        print(f"Error verifying file: {e}")
        return False

# ================= MAIN EXECUTION =================
if __name__ == "__main__":
    print("=" * 60)
    print("PROCESSING RAW .MAT FILES - MOTOR IMAGERY DATA ONLY")
    print("=" * 60)
    
    # Test with first 2 subjects
    print("\nTesting with first 2 subjects...")
    test_results = process_all_subjects(raw_data_dir, output_dir, max_subjects=2)
    
    # Verify the saved files
    if test_results:
        print("\n" + "=" * 60)
        print("VERIFYING SAVED FILES")
        print("=" * 60)
        
        for subject_id in list(test_results.keys())[:2]:
            verify_saved_file(subject_id, output_dir)
    
    # Ask if user wants to process all subjects
    print("\n" + "=" * 60)
    response = input("Process ALL subjects? (y/n): ")
    
    if response.lower() == 'y':
        print("\nProcessing all subjects...")
        all_results = process_all_subjects(raw_data_dir, output_dir)
        
        print("\n" + "=" * 60)
        print("PROCESSING COMPLETE!")
        print("=" * 60)
        print(f"Results saved in: {output_dir}")
        print(f"Files created:")
        
        # List created files
        created_files = os.listdir(output_dir)
        for file in sorted(created_files):
            if file.endswith('.npz'):
                file_path = os.path.join(output_dir, file)
                file_size = os.path.getsize(file_path) / 1024 / 1024
                print(f"  {file} ({file_size:.2f} MB)")
    
    else:
        print("Processing stopped. Only test files were created.")

Channels All: 33
Channels EEG: 30
Channels EOG: 2
Channels Marker: 1
PROCESSING RAW .MAT FILES - MOTOR IMAGERY DATA ONLY

Testing with first 2 subjects...
Found 2 raw .mat files
Output directory: D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_labelled_results

Processing subject: 01
  ✓ Loaded data: (40, 33, 4000)
  ✓ Labels: (40,)
  ✓ Samples per trial breakdown:
    - Instruction (first 2s): 1000 samples
    - Motor Imagery (middle 4s): 2000 samples
    - Break (last 2s): 1000 samples
    - Total: 4000 samples
  ✓ Original data shape: (40, 33, 4000)
  ✓ MI data shape (all channels): (40, 33, 2000)
  ✓ MI data shape (EEG only): (40, 30, 2000)
  ✅ Saved to: D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_labelled_results\sub-01_motor_imagery.npz
  📊 Summary: 20 left hand, 20 right hand trials

Processing subject: 02
  ✓ Loaded data: (40, 33, 4000)
  ✓ Labels: (40,)
  ✓ Samples per trial breakdown:
    - Instruction (first 2s): 1000 sampl

In [3]:
import numpy as np
import os

# Path to the file
file_path = r"D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_labelled_results\sub-01_motor_imagery.npz"

# Load the file
data = np.load(file_path, allow_pickle=True)

print("=" * 60)
print(f"FILE: {os.path.basename(file_path)}")
print("=" * 60)

print("\n1. KEYS AVAILABLE IN THE FILE:")
print("-" * 40)
for key in data.files:
    print(f"  • {key}")

print("\n2. DETAILED INFORMATION:")
print("-" * 40)

for key in data.files:
    value = data[key]
    print(f"\n{key}:")
    if hasattr(value, 'shape'):
        print(f"  Shape: {value.shape}")
        print(f"  Data type: {value.dtype}")
        print(f"  Size in memory: {value.nbytes / 1024:.2f} KB")
        
        # Additional info for specific arrays
        if key in ['segments_mi_all', 'segments_mi_eeg']:
            print(f"  Interpretation:")
            print(f"    - Trials: {value.shape[0]}")
            print(f"    - Channels: {value.shape[1]}")
            print(f"    - Samples per trial: {value.shape[2]}")
            print(f"    - Duration per trial: {value.shape[2]/500:.2f} seconds (at 500Hz)")
    
    elif isinstance(value, np.ndarray) and value.dtype == object:
        # Handle object arrays (likely the event_info)
        print(f"  Type: {type(value)}")
        print(f"  Length: {len(value)}")
        if len(value) > 0:
            print(f"  First element type: {type(value[0])}")
            if isinstance(value[0], dict):
                print(f"  First element keys: {list(value[0].keys())}")
    
    elif isinstance(value, list):
        print(f"  Type: list")
        print(f"  Length: {len(value)}")
    
    elif isinstance(value, (str, int, float)):
        print(f"  Value: {value}")
        print(f"  Type: {type(value).__name__}")

print("\n3. SPECIFIC ANALYSIS:")
print("-" * 40)

# Check labels distribution
if 'labels' in data:
    labels = data['labels']
    print(f"\nLabels distribution:")
    print(f"  Total trials: {len(labels)}")
    print(f"  Left hand (1): {np.sum(labels == 1)} trials")
    print(f"  Right hand (2): {np.sum(labels == 2)} trials")
    print(f"  Unique values: {np.unique(labels)}")

# Check channels information
if 'channels_all' in data:
    channels_all = data['channels_all']
    print(f"\nChannels information:")
    print(f"  Total channels: {len(channels_all)}")
    print(f"  First 10 channels: {channels_all[:10]}")
    print(f"  Last 10 channels: {channels_all[-10:]}")
    
    # Check for duplicates
    from collections import Counter
    channel_counts = Counter(channels_all)
    duplicates = {k: v for k, v in channel_counts.items() if v > 1}
    if duplicates:
        print(f"  ⚠️ DUPLICATE CHANNELS FOUND: {duplicates}")
    else:
        print(f"  ✓ No duplicate channel names")

# Check EEG channels specifically
if 'channels_eeg' in data:
    channels_eeg = data['channels_eeg']
    print(f"\nEEG channels:")
    print(f"  Number of EEG channels: {len(channels_eeg)}")
    print(f"  EEG channels: {channels_eeg}")

# Check EOG channels
if 'channels_eog' in data:
    channels_eog = data['channels_eog']
    print(f"\nEOG channels:")
    print(f"  Number of EOG channels: {len(channels_eog)}")
    print(f"  EOG channels: {channels_eog}")

# Check event_info structure
if 'event_info' in data:
    event_info = data['event_info']
    print(f"\nEvent info structure:")
    print(f"  Number of events: {len(event_info)}")
    if len(event_info) > 0:
        first_event = event_info[0]
        print(f"  First event keys: {list(first_event.keys())}")
        print(f"  First event values: {first_event}")

# Check sampling rate
if 'sampling_rate' in data:
    print(f"\nSampling rate: {data['sampling_rate']} Hz")

# Check mi_info
if 'mi_info' in data:
    mi_info = data['mi_info'].item()  # Use .item() for scalar arrays
    print(f"\nMotor Imagery Info:")
    for k, v in mi_info.items():
        print(f"  {k}: {v}")

print("\n4. MEMORY USAGE SUMMARY:")
print("-" * 40)
total_memory = 0
for key in data.files:
    value = data[key]
    if hasattr(value, 'nbytes'):
        memory_kb = value.nbytes / 1024
        total_memory += memory_kb
        print(f"  {key:20s}: {memory_kb:8.2f} KB")
    elif isinstance(value, np.ndarray) and value.dtype == object:
        # Estimate memory for object arrays
        print(f"  {key:20s}: (object array)")
    else:
        print(f"  {key:20s}: (small)")

print(f"\n  {'Total':20s}: {total_memory:8.2f} KB ({total_memory/1024:.2f} MB)")

print("\n" + "=" * 60)
print("DATA SHAPE SUMMARY:")
print("=" * 60)
print(f"\nFor Motor Imagery Analysis:")
print(f"• segments_mi_all shape: {data['segments_mi_all'].shape}")
print(f"  - {data['segments_mi_all'].shape[0]} trials")
print(f"  - {data['segments_mi_all'].shape[1]} channels (all)")
print(f"  - {data['segments_mi_all'].shape[2]} samples per trial")
print(f"  - {data['segments_mi_all'].shape[2]/500:.2f} seconds per trial")
print(f"\n• segments_mi_eeg shape: {data['segments_mi_eeg'].shape}")
print(f"  - {data['segments_mi_eeg'].shape[0]} trials")
print(f"  - {data['segments_mi_eeg'].shape[1]} EEG channels only")
print(f"  - {data['segments_mi_eeg'].shape[2]} samples per trial")
print(f"  - {data['segments_mi_eeg'].shape[2]/500:.2f} seconds per trial")

# Close the file
data.close()

FILE: sub-01_motor_imagery.npz

1. KEYS AVAILABLE IN THE FILE:
----------------------------------------
  • segments_mi_all
  • segments_mi_eeg
  • labels
  • channels_all
  • channels_eeg
  • channels_eog
  • channels_marker
  • sampling_rate
  • subject_id
  • event_info
  • mi_info

2. DETAILED INFORMATION:
----------------------------------------

segments_mi_all:
  Shape: (40, 33, 2000)
  Data type: float32
  Size in memory: 10312.50 KB
  Interpretation:
    - Trials: 40
    - Channels: 33
    - Samples per trial: 2000
    - Duration per trial: 4.00 seconds (at 500Hz)

segments_mi_eeg:
  Shape: (40, 30, 2000)
  Data type: float32
  Size in memory: 9375.00 KB
  Interpretation:
    - Trials: 40
    - Channels: 30
    - Samples per trial: 2000
    - Duration per trial: 4.00 seconds (at 500Hz)

labels:
  Shape: (40,)
  Data type: int32
  Size in memory: 0.16 KB

channels_all:
  Shape: (33,)
  Data type: <U6
  Size in memory: 0.77 KB

channels_eeg:
  Shape: (30,)
  Data type: <U3
  Siz